In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from sklearn.cluster import KMeans


import random 
from random import seed

import time


import numpy as np
import matplotlib.pyplot as plt

from lib import Kmeans_lib
from lib.Kmeans_lib import *
from lib.EvalMetrics import *


In [ ]:

# Load features and labels
features_saved = np.loadtxt('Models/Original_model/ll_features_20.txt')
labels_features_saved = np.loadtxt('Models/Original_model/ll_labels_features_20.txt').astype(int)


# Load model - CFR
model = keras.models.load_model('Models/Original_model/mnist_cnn.h5') # Frozen model 

# model_frozen = keras.models.Sequential(model.layers[:-1])  # extract the last layer from the original model
# model_frozen.compile()

model_frozen = keras.models.load_model('Models/Frozen_model/omv_mnist_cnn.h5')

# Print
# model.summary()
# model_frozen.summary()

# Absolute path is needed to load libraries 
ROOT_PATH = os.path.abspath('')
sys.path.append(ROOT_PATH + '/lib')



# Test import
# from lib import simulation_lib
# from lib.simulation_lib import

In [ ]:
import numpy as np
import cv2
import serial.tools.list_ports
import serial, struct
import pandas as pd
from PIL import Image
import tensorflow as tf
from tensorflow import keras

import time
import glob
from keras import applications
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, f1_score

import seaborn as sns
import re
from random import seed

from numpy.ma.core import size

import sys, os

ROOT_PATH = os.path.dirname(os.path.abspath(__file__))
sys.path.insert(0, ROOT_PATH + '/lib')

from importMnist import createDataset

# --------------------------------------------------------------------------
#    _______  ______  _        _    _   _    _  _____ ___ ___  _   _
#   | ____\ \/ /  _ \| |      / \  | \ | |  / \|_   _|_ _/ _ \| \ | |
#   |  _|  \  /| |_) | |     / _ \ |  \| | / _ \ | |  | | | | |  \| |
#   | |___ /  \|  __/| |___ / ___ \| |\  |/ ___ \| |  | | |_| | |\  |
#   |_____/_/\_\_|   |_____/_/   \_\_| \_/_/   \_\_| |___\___/|_| \_|

"""
This python script is used for sincronizing the OpenMV camera and the laptop during training. The idea is to disaply digits images on the 
laptop screen and at the same time send throught the UART (usb cable) to the OpenMV camera the correct label of the image displayed.
This should allow the camera to have the true label and correctly compute the error and later perform the backpropagation on biases and weights 
in order to perform the OL training.

Note that the UART on the USB cable is usually occupied by the OpenMV IDE, which will use this cable for receiving all the debugging informations from
the OpenMV camera (such as the video stream). In order to be able to communicate the informations from the Laptop to the OpenMV camera it is necessary to
flash the MicroPython code on the camera as the main.py script (in the IDE go to Tools->Save opened scipt as main.py). In this way, any time the camera
is powered on and NOT connected in debugging mode to the IDE (in the IDE in the bottom left corner you should see the disconnected image), the main.py script
is ran automatically and is possible to use the UART connection for sending and receiveing data (also the photos taken from the camera can be sent to the 
PC but for sure the code will be slower and it can easily get out of sync).
"""


############################################################
#    _____ _   _ _   _  ____ _____ ___ ___  _   _ ____
#   |  ___| | | | \ | |/ ___|_   _|_ _/ _ \| \ | / ___|
#   | |_  | | | |  \| | |     | |  | | | | |  \| \___ \
#   |  _| | |_| | |\  | |___  | |  | | |_| | |\  |___) |
#   |_|    \___/|_| \_|\____| |_| |___\___/|_| \_|____/


##################################################################
# DEFINITION OF THE FUNCTIONS USED FOR THE OPENMV
##################################################################

def on_change(val):
    # Function called when the sliding bar changes value.                                            #
    # The function changes the value TRAINING FLAG that is used as a indicator of the state machine. #
    # Parameters                                                                                     #
    # ..........                                                                                     #
    # val : integer                                                                                  #
    # Is the value set by the sliding bar                                                            #

    myClass.TRAINING_FLAG = val
    if (val == 0):
        print('Script is in IDLE MODE')
    elif (val == 1):
        print('Script is in STREAMING MODE')
    elif (val == 2):
        print('Script is in STREAMING ELABORATION MODE')
    elif (val == 3):
        print('Script is in TRAINING MODE')



class uselessContainer():
    # Container that I use because I need to change the parameter TRAINING_FLAG            #
    # if I don't use a class the value is not changed by ID and the script never           #
    # updates the real value but creates a new value with the same name but different ID   #
    
    def __init__(self):
        self.TRAINING_FLAG = 3
        self.cont = 0 

##################################################################

def k_mean_clustering(features_x_test, features_init, y_test, labels_features_init, n_cluster, batch_size):

    cntr = 0
    err = 0

    n_count = np.zeros((batch_size, n_cluster)) #inizializzo un vettore che ci permette di contare per ogni cluster quali numeri sono stati inseriti
    pseudo_label = np.zeros(batch_size)
 
    # creo due vettori vuoti uno per le label lungo (n_features) l'altro per le features lungo (n_features,512)
    features = np.zeros((int(n_cluster*len(labels_features_init)/10),512))
    labels_features = np.zeros(int(n_cluster*len(labels_features_init)/10))

    #passo il vettore salvato nel file e mi tiro fuori solo i numeri che mi interessano. Se ho 6 cluster, mi interessano solo gli 0, 1, 2, 3, 4 e 5.
    for i in range(0,len(features_init)-1):
        if labels_features_init[i] < n_cluster:
            features[cntr,:] = features_init[i,:]
            labels_features[cntr] = labels_features_init[i]
            cntr += 1

    # aggiungo la feature che di cui ci interessa trovare la pseudo label
    features = np.concatenate((features, features_x_test))
    labels_features = np.append(labels_features, y_test)

    # viene fatto il clustering
    k_mean = create_k_mean(features ,n_cluster)

    # qui viene riempito il vettore conteggio. Se l'immagine che ci interessa classificare è stata inserita nel cluster 0, il ciclo guarderà 
    # tutte le labels delle immagini che sono state messe in quel cluster e la label che compare più volte sarà la pseudo_label della nostra immagine
    for j in range(1, batch_size+1):
        for i in range(0,len(k_mean.labels_) - batch_size):
            if k_mean.labels_[i] == k_mean.labels_[-j]:
                if labels_features[i] == 0:
                    n_count[-j,0] +=1
                if labels_features[i] == 1:
                    n_count[-j,1] +=1
                if labels_features[i] == 2:
                    n_count[-j,2] +=1
                if labels_features[i] == 3:
                    n_count[-j,3] +=1
                if labels_features[i] == 4:
                    n_count[-j,4] +=1
                if labels_features[i] == 5:
                    n_count[-j,5] +=1
                if labels_features[i] == 6:
                    n_count[-j,6] +=1
                if labels_features[i] == 7:
                    n_count[-j,7] +=1
                if labels_features[i] == 8:
                    n_count[-j,8] +=1
                if labels_features[i] == 9:
                    n_count[-j,9] +=1


    # We extract the label for the new element
    for i in range(0,batch_size):
        pseudo_label[i]  =   np.argmax(n_count[i,:])
        pseudo_label[i] = pseudo_label[i].astype(int)  
        if pseudo_label[i]!= labels_features[i+int(n_cluster*len(labels_features_init)/10)]:
            err += 1

    return pseudo_label, err


def trainOneEpoch_CWR(model, x_test, y_test, features, labels_features, clust_batch_size, found_digit):
    
    learn_rate = model.l_rate
    model_batch_size = model.batch_size

    test_samples = x_test.shape[0]
    
    n_cluster = 10
    
    err_clu = 0
    err_mod = 0
    
    prediction_vec = np.zeros(test_samples)

    # CLUSTERING
    print('**********************************\n Performing clustering\n')
    
    # Pseudo-labels

    pseudo_labels, err_clu = k_mean_clustering(x_test, features, y_test, labels_features, n_cluster, clust_batch_size)

    # ONLINE-LEARNING
    print('**********************************\nPerforming training CWR \n ')  
                   
    # Cycle over all input samples
    for i in range(0, test_samples):
            
        CheckLabelKnown(model, pseudo_labels[i])
        y_true_soft = NumberToSoftmax(pseudo_labels[i], model.label)
        
        h = model.W.shape[0]
        w = model.W.shape[1] 

        found_digit[np.argmax(y_true_soft)] += 1  # update the digit counter
            
        # PREDICTION

        y_pred_c = softmax(np.array(np.matmul(x_test[i,:], model.W) + model.b))      
        y_pred_t = softmax(np.array(np.matmul(x_test[i,:], model.W_2) + model.b_2)) 
        
        prediction_vec[i] = np.argmax(y_pred_c)

        # Error
        if(prediction_vec[i] !=  y_test[i]):
        #if(np.argmax(y_pred) !=  y_test):  
            err_mod += 1

        # BACKPROPAGATION
        cost = y_pred_t-y_true_soft

        # Update weights
        for j in range(0,h):
            deltaW = np.multiply(cost, x_test[i,j])
            dW = np.multiply(deltaW, learn_rate)
            model.W_2[j,:] = model.W_2[j,:] - dW

        # Update biases
        db = np.multiply(cost, learn_rate)
        model.b_2 = model.b_2-db
        
        
        # If beginning of batch
        if(i%model_batch_size==0 and i!=0): 
            for k in range(0, w):
                if(found_digit[k]!=0):
                    tempW = np.multiply(model.W[:,k], found_digit[k])
                    tempB = np.multiply(model.b[k]  , found_digit[k])
                    model.W[:,k] = np.multiply(tempW+model.W_2[:,k], 1/(found_digit[k]+1))
                    model.b[k]   = np.multiply(tempB+model.b_2[k],   1/(found_digit[k]+1))
                    
            model.W_2  =  np.copy(model.W) # np.zeros((model.W.shape)) 
            model.b_2  =  np.copy(model.b) # np.zeros((model.b.shape))       
            found_digit = np.zeros(10)  # reset
    
    return pseudo_labels, prediction_vec, err_clu, err_mod


def trainOneEpoch_LWF(model, x_test, y_test, features, labels_features, clust_batch_size):
    
    learn_rate = model.l_rate
    model_batch_size = model.batch_size

    test_samples = x_test.shape[0]
    
    n_cluster = 10
    
    err_clu = 0
    err_mod = 0
    
    lam  = 0
    cntr = 1
    
    found_digit = np.zeros(10)
    
    prediction_vec = np.zeros(test_samples)
      
    
    # CLUSTERING
    print('**********************************\n Performing clustering\n')
    
    # Pseudo-labels

    pseudo_labels, err_clu = k_mean_clustering(x_test, features, y_test, labels_features, n_cluster, clust_batch_size)

    # ONLINE-LEARNING

    print('**********************************\nPerforming training with LWF - STOCHASTIC\n ') 
    
    y_LWF = np.zeros([1, 8])    # Define container for LWF

    # DEFINE ORIGINAL WEIGHTS AND BIASES
    model.W_2 = np.copy(model.W)
    model.b_2 = np.copy(model.b)
         
    # Cycle over every sample
    for i in range(0, test_samples):
                
        CheckLabelKnown(model, pseudo_labels[i] )
        y_true_soft = NumberToSoftmax(pseudo_labels[i], model.label)
                
        h = model.W.shape[0]
        w = model.W.shape[1]

        # PREDICTION WITH CURRENT WEIGHTS
     
        y_pred = model.predict(x_test[i,:])

        # PREDICTION WITH LWF WEIGHTS
        y_LWF = myFunc_softmax(np.array(np.matmul(y_ML, model.W_2) + model.b_2))
        
        lam = 100/(100+cntr)   
        
        # BACKPROPAGATION        
        cost_norm = y_pred-y_true_soft
        cost_LWF  = y_pred-y_LWF


        for j in range(0,h):
            # Update weights
            deltaW_norm  = np.multiply(cost_norm,1-lam)
            deltaW_LWF   = np.multiply(cost_LWF, lam)
            dW           = np.multiply(deltaW_norm+deltaW_LWF, y_ML[0,j]*learn_rate)
            model.W[j,:] = model.W[j,:]-dW

        # Update biases
        db_norm = np.multiply(cost_norm, 1-lam)
        db_LWF  = np.multiply(cost_LWF, lam)
        db      = np.multiply(db_norm+db_LWF, learn_rate)
        model.b = model.b-db
        
        
        # if the train data is finished still train the model but save the results
        if(i>=train_samples):
            
            # Find the max iter for both true label and prediction
            if(np.amax(y_true_soft) != 0):
                max_i_true = np.argmax(y_true_soft)

            if(np.amax(y_pred) != 0):
                max_i_pred = np.argmax(y_pred)

            # Fill up the confusion matrix
            for k in range(0,len(model.label)):
                if(model.label[max_i_pred] == model.std_label[k]):
                    p = np.copy(k)
                if(model.label[max_i_true] == model.std_label[k]):
                    t = np.copy(k)

            model.conf_matr[t,p] += 1 
        
        print(f"\r    Currently at {np.round(np.round(cntr/tot_samples,4)*100,2)}% of dataset", end="")
        cntr +=1
    
    
    return pseudo_labels, prediction_vec, err_clu, err_mod

def trainOneEpoch_OL(model, x_test, y_test, features, labels_features, batch_size):
    
    learn_rate = model.l_rate

    test_samples = x_test.shape[0]
    
    n_cluster = 10
    
    err_clu = 0
    err_mod = 0
    
    max_iter = int(test_samples//batch_size)

    prediction_vec = np.zeros(test_samples)
      
    
    # CLUSTERING
    print('**********************************\n Performing clustering\n')
    
    # Pseudo-labels

    pseudo_labels, err_clu = k_mean_clustering(x_test, features, y_test, labels_features, n_cluster, batch_size)

    # ONLINE-LEARNING
    print('**********************************\n Performing training with OL\n')

    for i in range(0, test_samples):

        CheckLabelKnown(model, pseudo_labels[i])
    
        y_true_soft = NumberToSoftmax(pseudo_labels[i], model.label)
               
        # Prediction
        y_pred = model.predict(x_test[i,:])
        prediction_vec[i] = np.argmax(y_pred)
        
        # Error
        if(prediction_vec[i] !=  y_test[i]):
        #if(np.argmax(y_pred) !=  y_test):
            err_mod += 1
        
        # Backpropagation
        cost = y_pred-y_true_soft
        
        for j in range(0,model.W.shape[0]):

            # Update weights
            dW = np.multiply(cost, x_test[i,j]*learn_rate)
            model.W[j,:] = model.W[j,:]-dW

        # Update biases
        db      = np.multiply(cost, learn_rate)
        model.b = model.b-db

    
    #y_true_soft = NumberToSoftmax(y_test, model.label)
                   
    # Find the max iter for both true label and prediction
    #if(np.amax(y_true_soft) != 0):
    #    max_i_true = np.argmax(y_true_soft)

    #if(np.amax(y_pred) != 0):
    #    max_i_pred = np.argmax(y_pred)

    # Fill up the confusion matrix
    #for k in range(0,len(model.label)):
    #    if(model.label[max_i_pred] == model.std_label[k]):
    #        p = np.copy(k)
    #    if(model.label[max_i_true] == model.std_label[k]):
    #        t = np.copy(k)

    #model.conf_matr[t,p] += 1 
    
    return pseudo_labels, prediction_vec, err_clu, err_mod




###################################
#    __  __    _    ___ _   _
#   |  \/  |  / \  |_ _| \ | |
#   | |\/| | / _ \  | ||  \| |
#   | |  | |/ ___ \ | || |\  |
#   |_|  |_/_/   \_\___|_| \_|

# Loop for saving new features
#for z in range(0,10):

#################################################
# PARAMETERS FOR IMPLEMENTATION OF ACTIVE MODEL
#################################################

# Path of the images to open
ROOT_PATH = os.path.dirname(os.path.abspath(__file__))

myClass = uselessContainer()  # Init the class that stores the state of the camera

# Open serial port
# Next lines are taken from the example script in the OpenMV IDE - the example is in    File->Examples->OpenMV->Board Control->usb_vcp.py
# NB: see the name of the com port used from the camera in                              Windows->Device manager->Ports(COM and LPT)
port = '/dev/tty.usbmodem3067376B30301'
sp = serial.Serial(port, baudrate=115200, bytesize=serial.EIGHTBITS, parity=serial.PARITY_NONE, xonxoff=False,
                rtscts=False, stopbits=serial.STOPBITS_ONE, timeout=5000, dsrdtr=True)
sp.setDTR(True)

# Import the dataset that I am going to display
samples_for_each_digit = 400
digits_i_want = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
#digits_i_want = [z]

digits_data, digits_label = createDataset(samples_for_each_digit, digits_i_want)  # load dataset ; originally was createDataset(samples_for_each_digit + 1, digits_i_want)
tot_samples = len(digits_label)                                                   # original is len(digits_label) - 8

testing_number = 500
training_number = tot_samples - testing_number

#########################################################
# EXTRA PARAMETERS FOR IMPLEMENTATION OF ACTIVE MODEL
#########################################################

ref_feat = np.loadtxt('ll_feat_50.txt')

labels_features = np.loadtxt('ll_lab_feat_50.txt')

model = keras.models.load_model('mnist_cnn.h5')

out_collect = []

batch_size = 50
err_cluster = 0
err_model = 0
found_digit = np.zeros(10)
cntr = 1
cntr_batch = 1

Model_OL = Custom_Layer(model)
Model_OL.title      = 'OL'
Model_OL.filename   = 'OL'
Model_OL.W_2  = np.zeros((Model_OL.W.shape))
Model_OL.b_2  = np.zeros((Model_OL.b.shape))
#Model_OL.W_2  = np.copy((Model_OL.W))
#Model_OL.b_2  = np.copy((Model_OL.b))
Model_OL.l_rate     = 0.01
Model_OL.batch_size = 10


######################
# START OF THE CODE
######################

print('\n\n ***** EVERYTHING IS LOADED - READY TO RUN ***** \n\n')

while 1:
    print('\n now i am in the loop\n')

    # Show digit/idle message
    if (myClass.TRAINING_FLAG != 0):
        zoom_digit = cv2.resize(digits_data[cntr-1], (0, 0), fx=7, fy=7)
        cv2.imshow('SYNC APP', zoom_digit)
        cv2.waitKey(150)


    # Send cmd + label to OpenMV
    if (myClass.TRAINING_FLAG == 0 or myClass.TRAINING_FLAG == 1):
        sp.write(b"snap")  # the camera will be in streaming mode
        sp.flush()

        # Receive image from OpenMV - careful it's easy to get out of sync
        size = struct.unpack('<L', sp.read(4))[0]
        img_raw = sp.read(size)
        img_openmv = cv2.imdecode(np.frombuffer(img_raw, np.uint8), cv2.IMREAD_COLOR)
        zoom_openmv = cv2.resize(img_openmv, (0, 0), fx=5, fy=5)
        cv2.imshow('OpenMV view - Zoomed', zoom_openmv)
        cv2.waitKey(150)


    elif (myClass.TRAINING_FLAG == 2):
        sp.write(b"elab")  # the camera will be in streaming mode
        sp.flush()
        
        # Receive image from OpenMV - careful it's easy to get out of sync
        size = struct.unpack('<L', sp.read(4))[0]
        img_raw = sp.read(size)
        img_openmv = cv2.imdecode(np.frombuffer(img_raw, np.uint8), 0)
        zoom_openmv = cv2.resize(img_openmv, (400, 400))
        cv2.imshow('OpenMV view - Zoomed', zoom_openmv)
        cv2.waitKey(150)

        
    elif (myClass.TRAINING_FLAG == 3):

        if cntr == 1:
            cv2.waitKey(8000)
        
        sp.write(b"trai")  # the camera will train on the image taken
        sp.flush()

        print(f'counter: ',cntr,'/',tot_samples,'\n')

        feature = sp.read(3072).decode("utf-8")
        sp.flush()

        # This section is a post processing in case of firmware.bin implementation on OpenMV.      #
        # Sometimes the converted features are bigger than 512 and I have to remove elements.      #
        # I choose void strings and strings containing only integer elements inside to be removed  #
        
        feature = feature[0:len(feature)-2].split(',')
        feature_new = (np.asarray([float(i) for i in feature])).reshape(1,len(feature))

        # This is section is for generating new features                                           #
        # The vector of the current feature is saved in a vector that contains, at the end of the  #
        # all the fetures of the same digit.                                                       #    
        
        #if cntr == 1:
        #    feature_file_0 = np.copy(feature_new)
        #    label_file_0 = np.copy(digits_label[cntr-1])
        #if cntr > 1:
        #    feature_file_0 = np.concatenate((feature_file_0,feature_new))
        #    label_file_0 = np.append(label_file_0,digits_label[cntr-1])
        
        # Generation of the batch vector   
                                                            
        if cntr_batch == 1:
            feature_batch = np.copy(feature_new)
            label_batch= np.copy(digits_label[cntr-1])
        if cntr_batch > 1:
            feature_batch= np.concatenate((feature_batch,feature_new))
            label_batch = np.append(label_batch,digits_label[cntr-1])

        # Elaboration
        if(cntr_batch == batch_size):
            
            # Make labels integers and rounds to 3 decimals the features
            label_batch = label_batch.astype(int)
            labels_features = labels_features.astype(int)
            #feature_batch = np.round(feature_batch,3)
            
            #pseudo_labels, predictions, err_clu, err_mod = trainOneEpoch_OL(Model_OL, feature_batch, label_batch, ref_feat, labels_features, batch_size)
            pseudo_labels, predictions, err_clu, err_mod = trainOneEpoch_CWR(Model_OL, feature_batch, label_batch, ref_feat, labels_features, batch_size, found_digit)

            # Error calculation
            if cntr > training_number:
                err_cluster = err_cluster + err_clu
                err_model = err_model + err_mod

            # Counter reset
            cntr_batch = 0

            print('\n*******************************************************************************')
            print('***** Currently at ', int(cntr/tot_samples*100), '%')
            print('***** Pseudo_labels: ', pseudo_labels.astype(int))
            print('***** Predictions:   ', predictions.astype(int))
            print('***** True labels:   ', label_batch)
            print('\n*******************************************************************************\n')      
            
        # Counters Update
        cntr_batch +=1
        cntr += 1 
    
    # Condition for exiting the loop at end training
    if (cntr == tot_samples + 1):
        sp.write(b"endt")
        myClass.TRAINING_FLAG = 5
        myClass.cont += 1
    
    if (myClass.cont == 1):
        break

print('\n*******************************************************************************')
print('***** Percentual error clustering committed ', int(err_cluster/testing_number*100),'%')
print('\n*******************************************************************************\n')

print('\n*******************************************************************************')
print('***** Percentual error model committed ', int(err_model/testing_number*100),'%')
print('\n*******************************************************************************\n')


# This is section is for generating new features                                           #
# The vector of the current feature is saved in a vector that contains, at the end of the  #
# all the fetures of all digits.                                                           #

#if z == 0:
#    feature_file = np.copy(feature_file_0)
#    label_file = np.copy(label_file_0)
#else:
#    feature_file = np.concatenate((feature_file,feature_file_0))
#    label_file = np.append(label_file,label_file_0)

print('*******************************************************************************')
print('***** The training images are finished, press ANY KEY to close the script *****')
print('*******************************************************************************')

#np.savetxt('ll_feat_100.txt', feature_file, fmt='%.3f')
#np.savetxt('ll_lab_feat_100.txt', label_file.astype(int),fmt='%i')